In [1]:
# !pip install -q -U google-genai pydantic

# Building Persona Script Generator with Gemini API
# Install required packages first

# !pip install -q -U google-genai

from google import genai
import os
import json
from datetime import datetime
from typing import Dict, List, Optional
import time

In [2]:
# Initialize the model

client = genai.Client()

# response = client.models.generate_content(
#     model="gemini-3-flash-preview",
#     contents="How does AI work?"
# )
# print(response.text)

## Use database to generate prompt

In [3]:
# Database file path
DB_FILE = "buildings_database.json"

def load_database() -> Dict:
    """Load the buildings database from file."""
    if os.path.exists(DB_FILE):
        with open(DB_FILE, 'r') as f:
            return json.load(f)
    return {}

def save_database(database: Dict) -> None:
    """Save the buildings database to file."""
    with open(DB_FILE, 'w') as f:
        json.dump(database, f, indent=2)

def get_building_data(building_name: str) -> Optional[Dict]:
    """Retrieve building data from database."""
    database = load_database()
    return database.get(building_name)

def update_building_data(building_name: str, data: Dict) -> None:
    """Update or create building data in database."""
    database = load_database()
    database[building_name] = data
    save_database(database)

## Generate building personas

In [14]:
from ast import Return


def get_persona(building_name: str) -> str:
    """
    Get or generate persona for a building.
    1. Check if building exists in database
    2. If it exists and has a persona, return it
    3. Otherwise, generate persona using Gemini API, store it, and return
    """
    building_data = get_building_data(building_name)
    
    if building_data and "persona" in building_data:
        return building_data["persona"]
    
    prompt = f"For the building named '{building_name}', based on its building type, historical facts and context, generate a persona using exactly 5 descriptive adjectives describing tone, speaking style, and personality traits."
    response = client.models.generate_content(
        model="gemini-3-flash-preview",
        contents=prompt
    )
    persona = response.text
    
    # Store persona in database
    if building_data is None:
        building_data = {}
    building_data["persona"] = persona
    update_building_data(building_name, building_data)
    
    return persona

## Prompt Gemini to get script

In [ ]:
def get_script(building_name: str) -> str:
    """
    Get or generate script for a building.
    1. Check if building exists in database
    2. If it exists and has persona and script, return script
    3. If it exists with persona but no script, generate and store it
    4. If it doesn't exist or has no persona, get persona first then generate script
    """
    building_data = get_building_data(building_name)
    
    if building_data and "script" in building_data:
        return building_data["script"]

    persona = building_data["persona"] if building_data and "persona" in building_data else get_persona(building_name)

    prompt = f"""Using the persona: {persona}
    
Generate a building script for '{building_name}' that:
- Reflects the persona defined above as if in conversation with a visitor
- Is approximately 2-3 sentences
- Begins with an attention-grabbing welcome statement to the building and area
- Highlights key historical facts and significance
- Emphasizes unique architectural features
- Conveys the building's role and importance in its location
- Includes the building's unique characteristics and stories"""
    
    response = client.models.generate_content(
        model="gemini-3-flash-preview",
        contents=prompt
    )
    script = response.text
    
    # Store script in database
    if building_data is None:
        building_data = {}
    building_data["script"] = script
    update_building_data(building_name, building_data)
    
    return script

In [21]:
def remove_persona(building_name: str) -> None:
    """Remove persona from building data."""
    building_data = get_building_data(building_name)
    if building_data and "persona" in building_data:
        del building_data["persona"]
        update_building_data(building_name, building_data)

def remove_script(building_name: str) -> None:
    """Remove script from building data."""
    building_data = get_building_data(building_name)
    if building_data and "script" in building_data:
        del building_data["script"]
        update_building_data(building_name, building_data)

In [ ]:
persona = get_persona("Empire State Building")
print(persona)

Majestic, resilient, sophisticated, authoritative, and eloquent.


In [24]:
remove_script("Empire State Building")
script = get_script("Empire State Building")
print(script)

Welcome to the beating heart of Manhattan, where I stand as a sovereign sentinel of the skyline, forged from the indomitable spirit of an era that dared to touch the heavens. Completed in a breathtaking thirteen months, my Art Deco silhouette and limestone grandeur remain an eternal testament to human ambition, famously crowned by a spire once envisioned as a docking port for transatlantic airships. I am more than a monument of steel; I am the very pulse of New York City, an unshakeable icon of elegance that has presided over this metropolis with majestic grace for generations.


In [25]:
scili = get_script("Brown University Science Library")
print(scili)

You stand before the Science Library, a fourteen-story concrete monolith that has commanded the Providence skyline since 1971 as the singular, unyielding vessel for the University’s empirical archives. Architect Warner Burns Toan Lunde’s brutalist design utilizes rigid geometry and repetitive verticality to centralize scientific inquiry, eschewing aesthetic distraction for the absolute efficiency of the reinforced structural frame. Known simply as the SciLi, this building functions as a heavy, waffle-slabbed anchor for Thayer Street, imposing the weight of objective truth upon the surrounding landscape.


In [26]:
whitehouse = get_script("The White House")